In [1]:
#%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import itertools as it
import ipyparallel as ipp

from random import normalvariate

import gc
import timeit
import sys
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

# Constructing hypothetical (synthetic) populations

In [4]:
# Create subdirectory /synthetic_data if it does not exist
path = "./synthetic_data"

# Check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

Folder ./synthetic_data already exists


In [5]:
# Setting up the cluster
cluster = ipp.Cluster(n=52)
cluster.start_cluster_sync()
rc = cluster.connect_client_sync()
rc.wait_for_engines(52)
rc.ids
dv = rc[:]

Using existing profile dir: '/Users/stigomholt/.ipython/profile_default'
Starting 52 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1739991800-zars', profile='default', controller=<running>, engine_sets=['1739991801'])>

  0%|          | 0/52 [00:00<?, ?engine/s]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51]

In [6]:
with dv.sync_imports():
    from random import normalvariate
    import numpy as np

importing normalvariate from random on engine(s)
importing numpy on engine(s)


In [7]:
a = 10
b = 5
dv.push({"a": a, "b": b})

<AsyncResult(_push): pending>

In [8]:
# To ensure the variables are on engine 0
rc[0]["a"]

10

In [9]:
rc[50]["b"]

5

### Calculate the initial sizes of all six population categories

In [10]:
%%px
def calculate_initial_size_of_each_category(population_data):
    total_number_calves = population_data[0] * population_data[3]
    total_number_adults = population_data[0] * (1 - population_data[3])
    total_number_stags = total_number_adults / (population_data[1] + 1)
    total_number_hinds = total_number_adults - total_number_stags

    female_calves_survivors = total_number_calves / 2
    male_calves_survivors = total_number_calves / 2

    young_hinds_survivors = total_number_hinds * population_data[4]
    adult_hinds_survivors = total_number_hinds - young_hinds_survivors

    young_stags_survivors = total_number_stags * population_data[5]
    adult_stags_survivors = total_number_stags - young_stags_survivors
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

### Calculate the winter survival of all six population categories
tested:
%%time
np.random.default_rng().normal(0.8, 0.04)
CPU times: user 270 µs, sys: 102 µs, total: 372 µs
Wall time: 356 µs

%%time
np.random.normal(0.8, 0.04)
CPU times: user 19 µs, sys: 9 µs, total: 28 µs
Wall time: 31 µs

%%time
normalvariate(0.8, 0.04)
CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 13.1 µs

When one samples only one normally distributed number, it seems that the built-in function normalvariate is dramatically faster than the new randomgenerator in numpy and more than twice as fast as the old numpy generator.
The built-in function is called by: 
from random import normalvariate

However, if one firste calls rng = np.random.default_rng(), and then
%%time
rng.normal(0.8, 0.04)
CPU times: user 19 µs, sys: 1e+03 ns, total: 20 µs
Wall time: 24.1 µs


Below the built-in function is used. The built-in is said to thread-safe: https://docs.python.org/3/library/random.html


In [11]:
%%px
def winter_mortality_all_categories(
    female_calves, young_hinds, adult_hinds, male_calves, young_stags, adult_stags
):
    # Assumed winter survival parameters for all six population categories
    w_survival = [
        normalvariate(0.8, 0.04),
        normalvariate(0.92, 0.03),
        normalvariate(0.93, 0.03),
        normalvariate(0.8, 0.04),
        normalvariate(0.87, 0.04),
        normalvariate(0.93, 0.03),
    ]
    if w_survival[0] >= 1.0:
        fraction_female_calves_surviving_winter = 1.0
    else:
        fraction_female_calves_surviving_winter = w_survival[0]

    if w_survival[1] >= 1.0:
        fraction_young_hinds_surviving_winter = 1.0
    else:
        fraction_young_hinds_surviving_winter = w_survival[1]

    if w_survival[2] >= 1.0:
        fraction_adult_hinds_surviving_winter = 1.0
    else:
        fraction_adult_hinds_surviving_winter = w_survival[2]

    if w_survival[3] >= 1.0:
        fraction_male_calves_surviving_winter = 1.0
    else:
        fraction_male_calves_surviving_winter = w_survival[3]

    if w_survival[4] >= 1.0:
        fraction_young_stags_surviving_winter = 1.0
    else:
        fraction_young_stags_surviving_winter = w_survival[4]

    if w_survival[5] >= 1.0:
        fraction_adult_stags_surviving_winter = 1.0
    else:
        fraction_adult_stags_surviving_winter = w_survival[5]

    female_calves_survivors = female_calves * fraction_female_calves_surviving_winter
    young_hinds_survivors = young_hinds * fraction_young_hinds_surviving_winter
    adult_hinds_survivors = adult_hinds * fraction_adult_hinds_surviving_winter

    male_calves_survivors = male_calves * fraction_male_calves_surviving_winter
    young_stags_survivors = young_stags * fraction_young_stags_surviving_winter
    adult_stags_survivors = adult_stags * fraction_adult_stags_surviving_winter

    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    )

### Spring update of population categories 

In [12]:
%%px
def spring_update_of_categories(
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
):
    # Surviving female calves become one year old young hinds
    young_hinds = female_calves_survivors

    # Surviving young hinds born almost two years ago become adult hinds
    adult_hinds = young_hinds_survivors + adult_hinds_survivors

    # Surviving male calves become one year old young stags
    young_stags = male_calves_survivors

    # Surviving young stags born almost two years ago become adult stags
    adult_stags = young_stags_survivors + adult_stags_survivors
    return young_hinds, adult_hinds, young_stags, adult_stags

### Calculate number of female and male calves born in early summer

In [13]:
%%px
def early_summer_breeding(young_hinds_survivors, adult_hinds_survivors):

    # Assumed fertility parameters for young and adult hinds
    calves_per_young_hind = normalvariate(0.4, 0.05)

    calves_per_adult_hind = normalvariate(0.975, 0.008)
    if calves_per_adult_hind >= 1.0:
        calves_per_adult_hind = 1.0

    female_calves_born = (
        young_hinds_survivors * calves_per_young_hind / 2
        + adult_hinds_survivors * calves_per_adult_hind / 2
    )
    male_calves_born = female_calves_born
    return (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Calculate putative emigration of stags

In [14]:
%%px
def emigration_of_stags(stags_per_hind_threshold, adult_stags, adult_hinds):
    """
    Does not account for the hypothesis that migration is primarily sported by
    the two years old stags. The way we deal with the emigration issue is highly
    idealized as we say that when the number of adult stags per adult hind is
    larger than a certain number, then stags will emigrate until the ratio falls
    below this number. The underlying hypothesis is that the stags are
    capable of making an assessment of their potential breeding success based on
    perceptual data during the winter and the spring and act on this information
    by use of a built in algorithm.
    """

    if stags_per_hind_threshold == 0 or adult_hinds < 1 or adult_stags < 1:
        number_of_migrants = 0
    else:
        number_of_migrants = adult_stags - stags_per_hind_threshold * adult_hinds
        if number_of_migrants <= 0:
            number_of_migrants = 0

    if number_of_migrants > 0:
        migration_percentage = 100 * number_of_migrants / adult_stags
    else:
        migration_percentage = 0

    adult_stags = adult_stags - number_of_migrants

    return adult_stags, number_of_migrants, migration_percentage

### Calculate number of individuals in each category surviving the hunting season

In [15]:
%%px
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

In [16]:
%%px
def survivors_after_hunting(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_born,
    male_calves_born,
    young_hinds,
    adult_hinds,
    young_stags,
    adult_stags,
):
    female_calves_survivors = female_calves_born - number_harvested_female_calves
    young_hinds_survivors = young_hinds - number_harvested_young_hinds
    adult_hinds_survivors = adult_hinds - number_harvested_adult_hinds

    male_calves_survivors = male_calves_born - number_harvested_male_calves
    young_stags_survivors = young_stags - number_harvested_young_stags
    adult_stags_survivors = adult_stags - number_harvested_adult_stags

    if female_calves_survivors < 0:
        female_calves_survivors = 0
    if young_hinds_survivors < 0:
        young_hinds_survivors = 0
    if adult_hinds_survivors < 0:
        adult_hinds_survivors = 0

    if male_calves_survivors < 0:
        male_calves_survivors = 0
    if young_stags_survivors < 0:
        young_stags_survivors = 0
    if adult_stags_survivors < 0:
        adult_stags_survivors = 0
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

### Project population development from one year to the next

In [17]:
%%px
def project_population_development(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
    stags_per_hind_threshold,
    year,
):

    # Let Mother winter take her toll
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    ) = winter_mortality_all_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

    # Let calves become young ones and young ones become adults
    young_hinds, adult_hinds, young_stags, adult_stags = spring_update_of_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )
    # Total population in the spring
    total_population_in_spring = young_hinds + adult_hinds + young_stags + adult_stags

    # Time for giving birth
    (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    ) = early_summer_breeding(young_hinds_survivors, adult_hinds_survivors)

    # A tough decision made by stags
    adult_stags, number_of_migrants, migration_percentage = emigration_of_stags(
        stags_per_hind_threshold, adult_stags, adult_hinds
    )

    # Calculate ratio hinds per stag just before hunting season
    if (young_stags + adult_stags) > 1.0:
        hinds_per_stag_before_hunting = (young_hinds + adult_hinds) / (
            young_stags + adult_stags
        )
    else:
        hinds_per_stag_before_hunting = "NaN"

    # Total population just before hunting season starts
    total_population_before_hunting = (
        female_calves_born
        + young_hinds
        + adult_hinds
        + male_calves_born
        + young_stags
        + adult_stags
    )

    # Oh Deer, life is becoming dangerous
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    ) = survivors_after_hunting(
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
        female_calves_born,
        male_calves_born,
        young_hinds,
        adult_hinds,
        young_stags,
        adult_stags,
    )

    # Total population in December after end of hunting season
    total_population_after_hunting = (
        female_calves_survivors
        + young_hinds_survivors
        + adult_hinds_survivors
        + male_calves_survivors
        + young_stags_survivors
        + adult_stags_survivors
    )

    # Calculate ratio hinds per stag in December after end of hunting season
    if (young_stags_survivors + adult_stags_survivors) > 1:
        hinds_per_stag_after_hunting = (
            young_hinds_survivors + adult_hinds_survivors
        ) / (young_stags_survivors + adult_stags_survivors)
    else:
        hinds_per_stag_after_hunting = "NaN"

    return (
        total_population_in_spring,
        number_of_migrants,
        migration_percentage,
        hinds_per_stag_before_hunting,
        total_population_before_hunting,
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        total_population_after_hunting,
        hinds_per_stag_after_hunting,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Transform harvest percentages to numbers

In [18]:
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

### Scripts for L2_norm_seen_deer analysis

In [19]:
%%px
def L2_norm_seen_deer(
    population_log,
    empirical_observations,
    number_of_years,
):
    """
    Calculating the minimum L2 norm between synthetic data and
    reported data
    """
    import numpy as np
    # Establish the two time series to be compared
    seen_deer_e = np.array(empirical_observations)
    before_hunt_record = [x[18] for x in population_log] #18:position tot_pop_bh
    seen_deer_p = np.array(before_hunt_record)
    
    if seen_deer_p[-1] > 10:
        # Find the scaling factor that minimises RSS and do the scaling
        scaling_factor = np.sum(np.multiply(seen_deer_p, seen_deer_e)) / np.sum(
            np.multiply(seen_deer_p, seen_deer_p)
        )
    else:
        scaling_factor = 100.
    
    seen_deer_p_scaled = scaling_factor * seen_deer_p

    # Find minimum Residual Sum Square value
    L2_norm_min = (np.sum((seen_deer_p_scaled - seen_deer_e) ** 2))**(1/2)

    return scaling_factor, L2_norm_min

### Main ipy parallel script

In [20]:
@dv.remote(block=True)
def ipy_master():
    import numpy as np

    # Initial preparations
    array_top_hits = np.zeros((number_of_top_hits, number_of_years, 34))
    new_val = np.arange(10000, 10000 + number_of_top_hits, 1)
    scaling_factor = 500
    for i in range(array_top_hits.shape[0]):
        array_top_hits[i, -1, array_top_hits.shape[2] - 2] = scaling_factor
        array_top_hits[i, -1, array_top_hits.shape[2] - 1] = new_val[i]

    # Start going through states
    for state in init_chunk:
        # Run through for replicate number z
        for z in range(number_of_replicates):
            replicate_number = z

            population_log = []

            # Calculate size of each category in the initial population December 31 year -1
            (
                female_calves_survivors,
                young_hinds_survivors,
                adult_hinds_survivors,
                male_calves_survivors,
                young_stags_survivors,
                adult_stags_survivors,
            ) = calculate_initial_size_of_each_category(state)

            # Record initial sizes of the six categories
            female_calves_initially = female_calves_survivors
            young_hinds_initially = young_hinds_survivors
            adult_hinds_initially = adult_hinds_survivors
            male_calves_initially = male_calves_survivors
            young_stags_initially = young_stags_survivors
            adult_stags_initially = adult_stags_survivors

            # Run through all years in the time series for each initial state
            for j in range(number_of_years):
                (
                    total_population_in_spring,
                    number_of_migrants,
                    migration_percentage,
                    hinds_per_stag_before_hunting,
                    total_population_before_hunting,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_after_hunting,
                    hinds_per_stag_after_hunting,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ) = project_population_development(
                    number_harvested_female_calves[j],
                    number_harvested_young_hinds[j],
                    number_harvested_adult_hinds[j],
                    number_harvested_male_calves[j],
                    number_harvested_young_stags[j],
                    number_harvested_adult_stags[j],
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    state[2],
                    j,
                )

                new_record = [
                    state[0],
                    state[1],
                    state[2],
                    female_calves_initially,
                    young_hinds_initially,
                    adult_hinds_initially,
                    male_calves_initially,
                    young_stags_initially,
                    adult_stags_initially,
                    replicate_number,
                    first_year + j,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_in_spring,
                    total_population_before_hunting,
                    total_population_after_hunting,
                    hinds_per_stag_before_hunting,
                    hinds_per_stag_after_hunting,
                    migration_percentage,
                    number_of_migrants,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ]

                population_log.append(new_record)

            # Do L2_norm fitting of the number_of_years population trajectory
            if municipality == "Surnadal":
                scaling_factor, L2_value = L2_norm_seen_deer(
                    population_log,
                    seen_deer_obs_farmland,
                    number_of_years,
                )
            else:
                scaling_factor, L2_value = L2_norm_seen_deer(
                    population_log,
                    seen_deer_obs,
                    number_of_years,
                )

            # Find max L2 norm in array_top_hits and exchange if better fit
            if L2_value < array_top_hits[np.argmax(array_top_hits[:, -1, -1]), -1, -1]:
                # Add scaling_factor and L2 value at the end of each record in population_log
                # and exchange number_of_years records
                for k in range(len(population_log)):
                    population_log[k].extend([scaling_factor, L2_value])
                    array_top_hits[
                        np.argmax(array_top_hits[:, -1, -1]), k, :
                    ] = population_log[k]

    # Make final sorting
    # array_top_hits = array_top_hits[array_top_hits[:,:,-1].argsort()]

    return array_top_hits

In [21]:
%%time
    
# Number of replicates of each initial condition 
number_of_replicates = 1 #50 #30
number_of_top_hits = 2000 # from each engine
dv.push(dict(number_of_replicates = number_of_replicates, number_of_top_hits = number_of_top_hits))

# To ensure the variables are on engines 
#print(rc[0]['number_of_replicates'],rc[50]['number_of_top_hits']) 

# Using all six municipalties
municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
data_municipality = [
    data_Averoy,
    data_Tingvoll,
    data_Surnadal,
    data_Sunndal,
    data_Vestnes,
    data_Laerdal,
]

for q in range(len(municipalities)):
    [
        municipality,
        first_year,
        last_year,
        number_of_years,
        years,
        seen_deer_obs,
        seen_deer_obs_forest,
        seen_deer_obs_farmland,
        hinds_per_stag_obs,
        total_harvest,
        total_quota,
        fraction_female_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_male_calves_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
        spring_counts,
    ] = data_municipality[q]
    
    dv.push(dict(municipality = municipality ,
            first_year = first_year,
            last_year = last_year,
            number_of_years = number_of_years,
            years = years,
            seen_deer_obs = seen_deer_obs,
            seen_deer_obs_forest = seen_deer_obs_forest,
            seen_deer_obs_farmland = seen_deer_obs_farmland,
            hinds_per_stag_obs = hinds_per_stag_obs,
            total_harvest = total_harvest,
            fraction_female_calves_harvested = fraction_female_calves_harvested,
            fraction_young_hinds_harvested = fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested = fraction_adult_hinds_harvested,
            fraction_male_calves_harvested = fraction_male_calves_harvested,
            fraction_young_stags_harvested = fraction_young_stags_harvested,
            fraction_adult_stags_harvested = fraction_adult_stags_harvested,
            spring_counts = spring_counts))

    # To ensure the variables are on engines 
    #print(rc[5]['years'],rc[45]['total_harvest']) 
    
    # Setting up all initial conditions to be run with calibrated init_pop
    # Make all combinations of the six initial conditions
    #initial_population = np.arange(init_pop_range[q][0],init_pop_range[q][1], 10)
    initial_population = np.arange(900, 4000, 10)
    #initial_hinds_per_stag = np.array([1.7]) # uses the inverse of the estimated stags_per_hind threshold
    initial_hinds_per_stag = np.arange(1.3, 2.2, 0.1)
    init_frac_calves = np.arange(0.2, 0.355, 0.01)
    init_frac_y_hinds = np.arange(0.09, 0.275, 0.01)  # of total number hinds
    init_frac_y_stags = np.arange(0.18, 0.545, 0.02)  # of total number stags
    #stags_per_hind_threshold = np.array([0.59])  # not really initial condition, but initial premise
    #stags_per_hind_threshold = np.arange(0.5, 0.8, 0.05)
    stags_per_hind_threshold = np.array([0, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80])

    all_initial_states = np.array(
        np.meshgrid(
            initial_population,
            initial_hinds_per_stag,
            stags_per_hind_threshold,
            init_frac_calves,
            init_frac_y_hinds,
            init_frac_y_stags,
        )
    ).T.reshape(-1, 6)

    dv.scatter("init_chunk", all_initial_states)
    # To ensure the variables are on engines 
    #check = rc[0]['init_chunk']
    #print("len(init_chunk):",len(check)) 
    
    # Transform reported hunting percentages for all years to numbers
    (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    ) = transform_harvested_percentages_to_numbers(
        total_harvest,
        fraction_female_calves_harvested,
        fraction_male_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
    )

    dv.push(dict(number_harvested_female_calves = number_harvested_female_calves,
        number_harvested_young_hinds = number_harvested_young_hinds,
        number_harvested_adult_hinds= number_harvested_adult_hinds,
        number_harvested_male_calves = number_harvested_male_calves,
        number_harvested_young_stags= number_harvested_young_stags,
        number_harvested_adult_stags = number_harvested_adult_stags))
    
    # To ensure the variables are on engines 
    #print(rc[10]['number_harvested_young_hinds'],rc[45]['number_harvested_adult_stags']) 
    
    # Getting array_top_hits from all engines in a 3d array that needs to be flattened
    array_top_hits = np.array(ipy_master())
    aggregated_arr = array_top_hits.reshape(-1, array_top_hits.shape[-1])
          

    # Make pandas dataframe for given municipality
    column_values = [
        "init_pop",
        "init_hps",
        "sh_threshold",
        "i_fcalves",
        "i_yhinds",
        "i_ahinds",
        "i_mcalves",
        "i_ystags",
        "i_astags",
        "rep_num",
        "obs_year",
        "f_calves",
        "y_hinds",
        "a_hinds",
        "m_calves",
        "y_stags",
        "a_stags",
        "spring_pop",
        "tot_pop_bh",
        "tot_pop_ah",
        "hps_bh",
        "hps_ah",
        "mig%",
        "num_migs",
        "ws_fc",
        "ws_yh",
        "ws_ah",
        "ws_mc",
        "ws_ys",
        "ws_as",
        "c_yh",
        "c_ah",
        "scaling",
        "L2_norm",
    ]
    pop_data = pd.DataFrame(data=aggregated_arr, columns=column_values)
    pop_data = pop_data.astype(
        {
            "init_pop": "int",
            "rep_num": "int",
            "obs_year": "int",
            "i_fcalves": "int",
            "i_yhinds": "int",
            "i_ahinds": "int",
            "i_mcalves": "int",
            "i_ystags": "int",
            "i_astags": "int",
            "f_calves": "int",
            "y_hinds": "int",
            "a_hinds": "int",
            "m_calves": "int",
            "y_stags": "int",
            "a_stags": "int",
            "spring_pop": "int",
            "tot_pop_bh": "int",
            "tot_pop_ah": "int",
            "num_migs": "int",
        }
    ).round(14)

    # Sort and trim the data frame
    #pop_data_sorted = pop_data.sort_values(by=["L2_norm"]).reset_index(drop=True)
    pop_data_sorted = pop_data.sort_values(['L2_norm','obs_year'],ascending=[True, True]).reset_index(drop=True)
        
    # Storing all top hits to allow variation in the number of top hits later on
    save_dir = "synthetic_data/"
    pop_data_sorted.to_pickle(
        save_dir + "top_hits_" + municipality + "_seen_deer_stoch_par" + ".pkl"
    )


    print("Municipality = ", municipality)
    pd.set_option('display.max_columns', None)
    display(pop_data_sorted)
    print()
    print()

Municipality =  Averoy


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1380,1.5,0.55,186,60,543,186,201,201,0,2007,212,125,506,208,145,279,1227,1675,1477,1.491061,1.490050,14.302119,50,0.842181,0.937678,0.902281,0.919435,0.855701,0.888337,0.350195,0.975736,0.000748,0.788789
1,1380,1.5,0.55,186,60,543,186,201,201,0,2008,220,114,544,222,115,298,1302,1743,1515,1.546324,1.592606,18.909802,75,0.739881,0.894580,0.938178,0.760422,0.918978,0.950009,0.460015,0.978364,0.000748,0.788789
2,1380,1.5,0.55,186,60,543,186,201,201,0,2009,223,93,531,226,166,326,1338,1825,1566,1.507855,1.268625,11.358323,42,0.768420,0.923285,0.925119,0.815247,0.825121,0.948213,0.417811,0.965537,0.000748,0.788789
3,1380,1.5,0.55,186,60,543,186,201,201,0,2010,211,103,492,209,133,274,1362,1705,1426,1.528186,1.462110,32.689991,149,0.787449,0.878817,0.895529,0.765718,0.878922,0.949589,0.351424,0.971890,0.000748,0.788789
4,1380,1.5,0.55,186,60,543,186,201,201,0,2011,202,88,485,202,130,269,1267,1648,1378,1.487887,1.435724,23.605773,92,0.745277,0.897243,0.916031,0.822875,0.900365,0.990133,0.389309,0.969774,0.000748,0.788789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1350,1.3,0.50,195,102,438,195,216,200,0,2019,181,85,426,184,108,213,1108,1532,1199,1.652245,1.586592,13.235259,39,0.828771,0.893458,0.946679,0.784748,0.884262,0.901350,0.464503,0.973238,0.000840,0.975798
1767996,1350,1.3,0.50,195,102,438,195,216,200,0,2020,144,41,360,154,89,193,1041,1403,985,1.593978,1.416485,18.554372,53,0.768087,0.891013,0.919071,0.797118,0.855308,0.908624,0.414766,0.979543,0.000840,0.975798
1767997,1350,1.3,0.50,195,102,438,195,216,200,0,2021,110,13,255,100,79,162,865,1155,722,1.596745,1.111034,22.683497,56,0.800176,0.892626,0.955958,0.780427,0.820762,0.892447,0.313843,0.970522,0.000840,0.975798
1767998,1350,1.3,0.50,195,102,438,195,216,200,0,2022,78,22,173,77,49,99,656,810,502,1.666666,1.316081,42.055449,95,0.796054,0.937405,0.977017,0.787728,0.836649,0.982726,0.480948,0.973692,0.000840,0.975798




Municipality =  Tingvoll


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,2870,1.3,0.8,416,253,898,416,301,584,0,2007,396,265,961,374,272,727,2540,3455,2998,1.242123,1.227328,0.000000,0,0.852146,0.888264,0.921557,0.803132,0.856630,0.925117,0.421929,0.990214,0.000154,0.188770
1,2870,1.3,0.8,416,253,898,416,301,584,0,2008,410,214,1033,386,240,834,2677,3634,3120,1.227338,1.161170,0.000000,0,0.791782,0.913253,0.955132,0.815660,0.790675,0.935962,0.311371,0.959349,0.000154,0.188770
2,2870,1.3,0.8,416,253,898,416,301,584,0,2009,402,229,984,398,248,826,2724,3644,3090,1.200752,1.130667,6.908882,67,0.781634,0.886731,0.908133,0.787220,0.875768,0.910238,0.385070,0.973481,0.000154,0.188770
3,2870,1.3,0.8,416,253,898,416,301,584,0,2010,328,222,876,328,262,749,2610,3344,2768,1.150146,1.086404,13.387016,126,0.754963,0.945135,0.820076,0.841660,0.808354,0.903024,0.385423,0.961453,0.000154,0.188770
4,2870,1.3,0.8,416,253,898,416,301,584,0,2011,367,174,873,357,210,726,2471,3228,2709,1.173230,1.117499,13.293571,123,0.786245,0.882766,0.926504,0.831144,0.877737,0.934543,0.460069,0.973144,0.000154,0.188770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,2500,1.8,0.5,375,269,855,375,337,287,0,2019,314,158,804,306,173,381,1998,2776,2137,1.688324,1.737633,10.219672,54,0.863671,0.880564,0.992518,0.806648,0.895264,0.970651,0.322087,0.961615,0.000187,0.228823
1767996,2500,1.8,0.5,375,269,855,375,337,287,0,2020,328,132,780,301,163,382,1981,2774,2089,1.645198,1.673343,9.275606,48,0.810685,0.929177,0.994865,0.839304,0.860145,0.979697,0.468384,0.964812,0.000187,0.228823
1767997,2500,1.8,0.5,375,269,855,375,337,287,0,2021,259,151,688,246,140,335,1843,2515,1820,1.691953,1.766230,13.917289,68,0.833167,0.920862,0.922287,0.790815,0.879659,0.903252,0.389033,0.962512,0.000187,0.228823
1767998,2500,1.8,0.5,375,269,855,375,337,287,0,2022,236,100,636,253,140,313,1627,2271,1681,1.647451,1.621568,9.751487,42,0.779940,0.886601,0.943832,0.839073,0.903617,0.917584,0.417182,0.970788,0.000187,0.228823




Municipality =  Surnadal


...



Municipality =  Vestnes


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,2220,1.4,0.65,288,182,776,288,355,328,0,2007,321,161,841,316,170,518,1972,2727,2328,1.371397,1.455482,5.311143,32,0.759473,0.932876,0.943501,0.801338,0.865386,0.947507,0.423652,0.978899,0.000224,0.361017
1,2220,1.4,0.65,288,182,776,288,355,328,0,2008,341,161,883,358,201,553,2150,2981,2500,1.334620,1.383414,4.730765,31,0.730815,0.929124,0.982430,0.862503,0.888451,0.992907,0.376431,0.976286,0.000224,0.361017
2,2220,1.4,0.65,288,182,776,288,355,328,0,2009,352,231,851,349,225,532,2246,3079,2543,1.382367,1.427716,4.819992,32,0.894122,0.949131,0.934583,0.816536,0.843310,0.900321,0.462528,0.962494,0.000224,0.361017
3,2220,1.4,0.65,288,182,776,288,355,328,0,2010,304,212,833,302,196,534,2193,2963,2383,1.395182,1.430199,6.746826,45,0.822708,0.914385,0.880866,0.778202,0.856894,0.895114,0.375936,0.980354,0.000224,0.361017
4,2220,1.4,0.65,288,182,776,288,355,328,0,2011,332,161,833,328,175,501,2015,2744,2333,1.373156,1.469102,8.170848,52,0.734877,0.921727,0.859978,0.774877,0.866720,0.889361,0.439448,0.970651,0.000224,0.361017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,2110,1.7,0.75,221,251,797,221,333,283,0,2019,296,159,751,262,160,502,1998,2761,2133,1.321588,1.375403,0.000000,0,0.842979,0.877558,0.939224,0.824951,0.848056,0.946273,0.426247,0.963767,0.000237,0.415571
1767996,2110,1.7,0.75,221,251,797,221,333,283,0,2020,270,177,711,262,96,540,1926,2660,2060,1.333900,1.394720,0.032959,0,0.881498,0.938435,0.917339,0.745418,0.922907,0.959207,0.437910,0.968441,0.000237,0.415571
1767997,2110,1.7,0.75,221,251,797,221,333,283,0,2021,260,124,658,244,109,489,1801,2504,1886,1.296606,1.308584,0.000000,0,0.771630,0.861771,0.920730,0.776778,0.818275,0.926785,0.365642,0.987830,0.000237,0.415571
1767998,2110,1.7,0.75,221,251,797,221,333,283,0,2022,273,162,638,255,147,469,1720,2392,1946,1.238159,1.299317,0.755754,4,0.797660,0.853422,0.964625,0.861126,0.832932,0.959456,0.483095,0.984417,0.000237,0.415571




Municipality =  Laerdal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1760,1.3,0.50,211,75,680,211,151,430,0,2007,264,132,594,259,123,271,1545,1986,1646,1.664603,1.840350,35.257252,186,0.783643,0.900490,0.904500,0.796817,0.827567,0.935843,0.296959,0.986425,0.000118,0.140557
1,1760,1.3,0.50,211,75,680,211,151,430,0,2008,265,173,624,272,169,294,1487,2104,1800,1.611963,1.722502,0.643183,2,0.770794,0.891932,1.000000,0.817752,0.803502,0.955247,0.345377,0.971937,0.000118,0.140557
2,1760,1.3,0.50,211,75,680,211,151,430,0,2009,238,164,607,240,157,297,1520,2043,1704,1.637637,1.698135,15.488444,63,0.777446,0.905209,0.865674,0.746222,0.836804,0.922144,0.397149,0.968410,0.000118,0.140557
3,1760,1.3,0.50,211,75,680,211,151,430,0,2010,267,137,652,258,146,306,1547,2150,1767,1.662969,1.744516,9.861472,41,0.795697,0.936910,0.983561,0.791509,0.887057,0.933022,0.416462,0.971681,0.000118,0.140557
4,1760,1.3,0.50,211,75,680,211,151,430,0,2011,252,184,615,241,165,279,1549,2094,1739,1.652729,1.797310,14.349801,58,0.874480,0.957534,0.864468,0.830481,0.855447,0.916604,0.437472,0.966669,0.000118,0.140557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1880,1.6,0.55,300,188,597,300,265,226,0,2019,255,144,662,244,131,383,1683,2291,1820,1.576075,1.568783,12.664350,62,0.777130,0.904012,0.945213,0.722059,0.829249,0.923952,0.426922,0.959513,0.000108,0.158357
1767996,1880,1.6,0.55,300,188,597,300,265,226,0,2020,240,146,642,227,142,361,1609,2208,1761,1.552435,1.565866,9.459771,43,0.801461,0.914976,0.930419,0.828321,0.880333,0.885299,0.350670,0.965684,0.000108,0.158357
1767997,1880,1.6,0.55,300,188,597,300,265,226,0,2021,244,111,636,215,94,361,1599,2196,1664,1.586721,1.639244,13.004831,61,0.802046,0.945727,0.951494,0.798477,0.936628,0.942566,0.408451,0.985979,0.000108,0.158357
1767998,1880,1.6,0.55,300,188,597,300,265,226,0,2022,256,126,634,245,88,335,1497,2107,1686,1.618385,1.794969,8.610633,36,0.809479,0.907460,0.948405,0.789290,0.844422,0.951995,0.497243,0.987090,0.000108,0.158357




CPU times: user 6min 15s, sys: 11min 51s, total: 18min 7s
Wall time: 2h 48min 24s


In [22]:
# stop cluster syncronously
cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 10414, 'identifier': 'ipcontroller-1739991800-zars-10410'}
Stopping engine(s): 1739991801
engine set stopped 1739991801: {'engines': {'0': {'exit_code': 0, 'pid': 10427, 'identifier': '0'}, '1': {'exit_code': 0, 'pid': 10433, 'identifier': '1'}, '2': {'exit_code': 0, 'pid': 10434, 'identifier': '2'}, '3': {'exit_code': 0, 'pid': 10436, 'identifier': '3'}, '4': {'exit_code': 0, 'pid': 10438, 'identifier': '4'}, '5': {'exit_code': 0, 'pid': 10441, 'identifier': '5'}, '6': {'exit_code': 0, 'pid': 10444, 'identifier': '6'}, '7': {'exit_code': 0, 'pid': 10448, 'identifier': '7'}, '8': {'exit_code': 0, 'pid': 10452, 'identifier': '8'}, '9': {'exit_code': 0, 'pid': 10456, 'identifier': '9'}, '10': {'exit_code': 0, 'pid': 10462, 'identifier': '10'}, '11': {'exit_code': 0, 'pid': 10468, 'identifier': '11'}, '12': {'exit_code': 0, 'pid': 10474, 'identifier': '12'}, '13': {'exit_code': 0, 'pid': 10480, 'identifier': '13'}, '14': {'ex

In [23]:
cluster._is_running()

False

# Testing the sanity of the results

In [24]:
# Revoking top hits frames
save_dir = "synthetic_data/"
# With option for emigration
top_hits_Averoy_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Averoy_seen_deer_stoch_par.pkl"
)
top_hits_Tingvoll_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Tingvoll_seen_deer_stoch_par.pkl"
)
top_hits_Surnadal_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Surnadal_seen_deer_stoch_par.pkl"
)
top_hits_Sunndal_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Sunndal_seen_deer_stoch_par.pkl"
)
top_hits_Vestnes_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Vestnes_seen_deer_stoch_par.pkl"
)
top_hits_Laerdal_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Laerdal_seen_deer_stoch_par.pkl"
)

top_hits_seen_deer = [
    top_hits_Averoy_seen_deer,
    top_hits_Tingvoll_seen_deer,
    top_hits_Surnadal_seen_deer,
    top_hits_Sunndal_seen_deer,
    top_hits_Vestnes_seen_deer,
    top_hits_Laerdal_seen_deer,
]

In [25]:
save_dir = "synthetic_data/"
municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
for q in range(len(top_hits_seen_deer)):
    # Read through and test for consistent periodicity regarding years
    # which checks that there are no conflicts associated with the sorting of the data frames
    counter = 0
    for i in range(0, len(top_hits_seen_deer[q]), number_of_years):
        for j in range(number_of_years):
            if top_hits_seen_deer[q]["obs_year"][i + j] != (2007 + j):
                # print('i:', i, 'j:', j, 'obs_year value:', top_hits_seen_deer[q]['obs_year'][i+j])
                counter = counter + 1
    print("Number of troubles in " + municipalities[q] + ":", counter)

Number of troubles in Averoy: 0
Number of troubles in Tingvoll: 0
Number of troubles in Surnadal: 0
Number of troubles in Sunndal: 0
Number of troubles in Vestnes: 0
Number of troubles in Laerdal: 0


#pd.set_option("display.precision", 8)
pd.options.display.float_format = '{:.6f}'.format
top_hits_Averoy_seen_deer.dtypes
pd.set_option('display.max_columns', None)